# Importación de Librerías

In [1]:
import pandas as pd
from time import time
import math

# Preprocesamiento de Datos

### Carga de Datos

Cargamos nuestro listado de normas de Infoleg

In [2]:
start_time = time()
df = pd.read_csv('base-infoleg-normativa-nacional.csv', dtype='string').fillna('')
print ("Tiempo de importar csv: " + str(time() - start_time))
df.head()

Tiempo de importar csv: 2.829439878463745


,id_norma,tipo_norma,numero_norma,clase_norma,organismo_origen,fecha_sancion,numero_boletin,fecha_boletin,pagina_boletin,titulo_resumido,titulo_sumario,texto_resumido,observaciones,texto_original,texto_actualizado,modificada_por,modifica_a
0,183290,Ley,S/N,,HONORABLE CONGRESO DE LA NACION ARGENTINA,1853-11-09,,,,ORGANIZACION DE LA HACIENDA Y CREDITO PUBLICO,ADMINISTRACION PUBLICA NACIONAL,ESTATUTO PARA LA ORGANIZACION DE LA HACIENDA Y...,LA PRESENTE LEY CORRESPONDE A LAS SANCIONADAS ...,http://servicios.infoleg.gob.ar/infolegInterne...,,1,
1,233172,Decreto,S/N,,PODER EJECUTIVO NACIONAL (P.E.N.),1854-05-29,,,,UNIVERSIDAD MAYOR DE SAN CARLOS - ESTABLECESE ...,EDUCACION SUPERIOR,LA UNIVERSIDAD MAYOR DE SAN CARLOS EN LA CIUDA...,PUBLICADA EN REGISTRO OFICIAL DE LA REPUBLICA ...,http://servicios.infoleg.gob.ar/infolegInterne...,,0,
2,209422,Decreto,S/N,,PODER EJECUTIVO NACIONAL (P.E.N.),1854-09-26,,,,SE MANDA PREPARAR EL PRESUPUESTO GENERAL PARA ...,PRESUPUESTO GENERAL DE LA ADMINISTRACION NACIONAL,ART. 1° LOS MINISTROS DEL GOBIERNO DE LA CONFE...,LA NUMERACION DE LA PRESENTE LEY CORRESPONDE A...,http://servicios.infoleg.gob.ar/infolegInterne...,,3,
3,280916,Ley,1,,HONORABLE CONGRESO DE LA NACION ARGENTINA,1854-11-05,,,,CREDITO FISCAL,FISCO NACIONAL,OBJETO CUMPLIDO-DESMONETIZACION DE LOS BILLETE...,PUBLICADA EN REGISTRO NACIONAL DE 1856,,,0,
4,280915,Ley,2,,HONORABLE CONGRESO DE LA NACION ARGENTINA,1854-11-07,,,,,,AMBITO TEMPORAL CUMPLIDO-DERECHO ADICIONAL A L...,PUBLICADA EN REGISTRO NACIONAL DE 1856,,,0,


### Limpieza y Ordenamiento del dataset

##### Eliminamos las columnas que no nos importan y las filas que no tengan valor "texto_original" ya que es el link de donde sacaremos el texto 

In [3]:
dfclean = df[df['texto_original']!=""][["id_norma","tipo_norma","organismo_origen","fecha_sancion","texto_original"]].sort_values('fecha_sancion', ascending=False)
dfclean

,id_norma,tipo_norma,organismo_origen,fecha_sancion,texto_original
384513,385876,Resolución,SECRETARIA DE ENERGIA,2023-06-29,http://servicios.infoleg.gob.ar/infolegInterne...
384506,385845,Decreto,PODER EJECUTIVO NACIONAL (P.E.N.),2023-06-29,http://servicios.infoleg.gob.ar/infolegInterne...
384492,385871,Resolución,SECRETARIA DE COMERCIO,2023-06-29,http://servicios.infoleg.gob.ar/infolegInterne...
384500,385875,Resolución,SECRETARIA DE ENERGIA,2023-06-29,http://servicios.infoleg.gob.ar/infolegInterne...
384502,385874,Resolución,SECRETARIA DE ENERGIA,2023-06-29,http://servicios.infoleg.gob.ar/infolegInterne...
...,...,...,...,...,...
10,214371,Ley,HONORABLE CONGRESO DE LA NACION ARGENTINA,1854-11-28,http://servicios.infoleg.gob.ar/infolegInterne...
7,280913,Ley,HONORABLE CONGRESO DE LA NACION ARGENTINA,1854-11-21,http://servicios.infoleg.gob.ar/infolegInterne...
2,209422,Decreto,PODER EJECUTIVO NACIONAL (P.E.N.),1854-09-26,http://servicios.infoleg.gob.ar/infolegInterne...
1,233172,Decreto,PODER EJECUTIVO NACIONAL (P.E.N.),1854-05-29,http://servicios.infoleg.gob.ar/infolegInterne...


Eliminamos aquellos registros por tipo_norma que cuyo organismo_origen tenga una frecuencia menor al 5%

In [4]:
minimun_size = 0.05

lista_tiponorma = dfclean['tipo_norma'].unique().tolist()

dataframes_seleccionados = []

for tipo_norma in lista_tiponorma:

    organismos_por_tiponorma = dfclean[dfclean['tipo_norma']==tipo_norma]['organismo_origen'].value_counts(normalize=True)

    valores_a_eliminar = organismos_por_tiponorma[organismos_por_tiponorma > minimun_size].index

    dataframes_seleccionados.append(dfclean[(dfclean['tipo_norma'] == tipo_norma) & (dfclean['organismo_origen'].isin(valores_a_eliminar))])

dfclean = pd.concat(dataframes_seleccionados)

dfclean

,id_norma,tipo_norma,organismo_origen,fecha_sancion,texto_original
384411,385738,Resolución,ADMINISTRACION FEDERAL DE INGRESOS PUBLICOS,2023-06-23,http://servicios.infoleg.gob.ar/infolegInterne...
384355,385662,Resolución,ADMINISTRACION FEDERAL DE INGRESOS PUBLICOS,2023-06-22,http://servicios.infoleg.gob.ar/infolegInterne...
384378,385661,Resolución,ADMINISTRACION FEDERAL DE INGRESOS PUBLICOS,2023-06-22,http://servicios.infoleg.gob.ar/infolegInterne...
384306,385579,Resolución,ADMINISTRACION FEDERAL DE INGRESOS PUBLICOS,2023-06-21,http://servicios.infoleg.gob.ar/infolegInterne...
384316,385498,Resolución,ADMINISTRACION FEDERAL DE INGRESOS PUBLICOS,2023-06-21,http://servicios.infoleg.gob.ar/infolegInterne...
...,...,...,...,...,...
13077,294786,Decreto/Ley,PODER EJECUTIVO NACIONAL (P.E.N.),1943-08-24,http://servicios.infoleg.gob.ar/infolegInterne...
13070,294779,Decreto/Ley,PODER EJECUTIVO NACIONAL (P.E.N.),1943-08-13,http://servicios.infoleg.gob.ar/infolegInterne...
13065,232611,Decreto/Ley,PODER EJECUTIVO NACIONAL (P.E.N.),1943-08-06,http://servicios.infoleg.gob.ar/infolegInterne...
13051,294764,Decreto/Ley,PODER EJECUTIVO NACIONAL (P.E.N.),1943-06-28,http://servicios.infoleg.gob.ar/infolegInterne...


##### Analizamos la frecuencia de ambas variables

In [5]:
dfclean['tipo_norma'].value_counts()

Resolución                 31103
Decreto                    21576
Comunicación               12042
Disposición                 9285
Ley                         7601
Decisión Administrativa     5049
Nota Externa                 531
Decreto/Ley                  526
Decisión                     479
Directiva                    313
Acordada                     269
Instrucción                  240
Circular                     157
Acta                         140
Laudo                         14
Convenio                      14
Recomendación                 10
Acuerdo                        7
Nota                           6
Providencia                    4
Protocolo                      2
Interpretación                 1
Name: tipo_norma, dtype: Int64

In [6]:
dfclean['organismo_origen'].value_counts().head(25)

SECRETARIA DE TRABAJO                                26053
PODER EJECUTIVO NACIONAL (P.E.N.)                    23637
BANCO CENTRAL DE LA REPUBLICA ARGENTINA (B.C.R...    12042
HONORABLE CONGRESO DE LA NACION ARGENTINA             6067
ADMINISTRACION FEDERAL DE INGRESOS PUBLICOS           5213
JEFATURA DE GABINETE DE MINISTROS                     5049
DIRECCION DE NORMATIVA LABORAL                        1961
DIRECCION NACIONAL DE RELACIONES DEL TRABAJO          1780
DIRECCION NACIONAL DE RELACIONES Y REGULACIONE...     1724
AGENCIA NACIONAL DE SEGURIDAD VIAL                    1365
DIRECCION NACIONAL DE REGULACIONES DEL TRABAJO        1302
ADM.NAC.DE MEDICAMENTOS, ALIMENTOS Y TEC. MEDICA      1181
CONSEJO DEL MERCADO COMUN                              499
DIRECCION GENERAL DE ADUANAS                           400
COMISION DE COMERCIO DEL MERCOSUR                      376
SUP. DE ADM. DE FONDOS DE JUBILACIONES Y PENSI...      240
CORTE SUPREMA DE JUSTICIA DE LA NACION                 1

Definimos considerar los primeros 14 tipo_norma (hasta acta) y distribuir los organismos mas frecuentes en esos tipos de documento

In [7]:
selected_types = dfclean['tipo_norma'].value_counts().head(14)
selected_types

Resolución                 31103
Decreto                    21576
Comunicación               12042
Disposición                 9285
Ley                         7601
Decisión Administrativa     5049
Nota Externa                 531
Decreto/Ley                  526
Decisión                     479
Directiva                    313
Acordada                     269
Instrucción                  240
Circular                     157
Acta                         140
Name: tipo_norma, dtype: Int64

##### Realizamos un balanceo del dataset

Determinamos por tipo_norma cuantos registros necesitamos

In [8]:
target_size = 10000
minimun_size = 0.01
current_length = selected_types.sum()

# Calculamos el filtro
selected_types_filtered = selected_types[(selected_types / current_length) >= minimun_size]
selected_types_filtered_sqrt = selected_types_filtered.apply(math.sqrt)
filtersum = selected_types_filtered_sqrt.sum()

# Calculamos el restante
selected_types_rest = selected_types[(selected_types / current_length) < minimun_size]
rest = selected_types_rest.sum()

# Aplicamos la lógica para la selección final
selected_types_balanced = selected_types.apply(
    lambda r: round((target_size - rest) * (math.sqrt(r) / filtersum)) if (r / current_length) >= minimun_size else r
)

selected_types_type = selected_types.apply(lambda r: True if (r / current_length) >= minimun_size else False)

selected_types_balanced

Resolución                 1884
Decreto                    1569
Comunicación               1172
Disposición                1029
Ley                         931
Decisión Administrativa     759
Nota Externa                531
Decreto/Ley                 526
Decisión                    479
Directiva                   313
Acordada                    269
Instrucción                 240
Circular                    157
Acta                        140
Name: tipo_norma, dtype: int64

Realizamos el balanceo final del dataset

In [34]:
minimun_size = 0.01

dataframes_seleccionados = []

for tipo_norma in selected_types_type.index:

    df_seleccionado = dfclean[dfclean['tipo_norma'] == tipo_norma]

    if selected_types_type.loc[tipo_norma]:
        
        organimos_por_tiponorma = df_seleccionado['organismo_origen'].value_counts(normalize=True).apply(math.sqrt)
        
        organimos_balanceados = (selected_types_balanced[tipo_norma]*(organimos_por_tiponorma/organimos_por_tiponorma.sum())).apply(round)
        
        for organismo, cantidad in organimos_balanceados.items():
            # Filtramos el DataFrame para el organismo específico y tomamos los primeros 'cantidad' registros
            df_seleccionado2 = df_seleccionado[df_seleccionado['organismo_origen'] == organismo].head(cantidad)
            
            # Agregamos el DataFrame resultante a la lista
            dataframes_seleccionados.append(df_seleccionado2)
    
    else:
        dataframes_seleccionados.append(df_seleccionado)

dffinal = pd.concat(dataframes_seleccionados)

dffinal

,id_norma,tipo_norma,organismo_origen,fecha_sancion,texto_original
383669,384691,Resolución,SECRETARIA DE TRABAJO,2023-06-01,http://servicios.infoleg.gob.ar/infolegInterne...
383070,385906,Resolución,SECRETARIA DE TRABAJO,2023-05-18,http://servicios.infoleg.gob.ar/infolegInterne...
383069,385908,Resolución,SECRETARIA DE TRABAJO,2023-05-18,http://servicios.infoleg.gob.ar/infolegInterne...
383068,385905,Resolución,SECRETARIA DE TRABAJO,2023-05-18,http://servicios.infoleg.gob.ar/infolegInterne...
383066,385901,Resolución,SECRETARIA DE TRABAJO,2023-05-18,http://servicios.infoleg.gob.ar/infolegInterne...
...,...,...,...,...,...
104813,68625,Acta,GRUPO DEL MERCADO COMUN,2001-06-12,http://servicios.infoleg.gob.ar/infolegInterne...
104211,68123,Acta,GRUPO DEL MERCADO COMUN,2001-04-25,http://servicios.infoleg.gob.ar/infolegInterne...
104187,68122,Acta,GRUPO DEL MERCADO COMUN,2001-04-23,http://servicios.infoleg.gob.ar/infolegInterne...
93709,100921,Acta,GRUPO DEL MERCADO COMUN,1999-03-09,http://servicios.infoleg.gob.ar/infolegInterne...


In [35]:
for tipo_norma in selected_types_type.index:
    print(
        "# " + tipo_norma+
        "\n------------------\n"+
        dffinal[dffinal['tipo_norma']==tipo_norma]['organismo_origen'].value_counts().to_string()+
        "\n\n> Total: "+ str(dffinal[dffinal['tipo_norma']==tipo_norma]['organismo_origen'].value_counts().sum())+
        "\n\n"
    )

# Resolución
------------------
SECRETARIA DE TRABAJO                          1308
ADMINISTRACION FEDERAL DE INGRESOS PUBLICOS     576

> Total: 1884


# Decreto
------------------
PODER EJECUTIVO NACIONAL (P.E.N.)    1569

> Total: 1569


# Comunicación
------------------
BANCO CENTRAL DE LA REPUBLICA ARGENTINA (B.C.R...    1172

> Total: 1172


# Disposición
------------------
DIRECCION DE NORMATIVA LABORAL                       194
DIRECCION NACIONAL DE RELACIONES DEL TRABAJO         185
DIRECCION NACIONAL DE RELACIONES Y REGULACIONE...    182
AGENCIA NACIONAL DE SEGURIDAD VIAL                   162
DIRECCION NACIONAL DE REGULACIONES DEL TRABAJO       158
ADM.NAC.DE MEDICAMENTOS, ALIMENTOS Y TEC. MEDICA     149

> Total: 1030


# Ley
------------------
HONORABLE CONGRESO DE LA NACION ARGENTINA    619
PODER EJECUTIVO NACIONAL (P.E.N.)            312

> Total: 931


# Decisión Administrativa
------------------
JEFATURA DE GABINETE DE MINISTROS    759

> Total: 759


# Nota Externa
--

### Guardamos el dataset generado

In [36]:
dffinal.to_csv('test2.csv', index=True, sep='\t', encoding='utf-8')